# Perplexity Research

In this example, we will introduce the use of a custom function for researching in Perplexity.  The recipe also provides good examples of how to use the output of one node into the others and also how to reference an attribute in the response, like a dictionary (see the citations part in final_outputs).

This recipe will use a custom function for researching in Perplexity. You can register the function in the registry with the decorator (see below) or via: 

```python
from content_composer.registry import RegistryScope, get_registry
registry = get_registry()
registry.register(
    identifier="extract_file_content",
    function=extract_file_content,
    description="Extracts content from files using content_core library",
    tags=["file", "extraction", "content"]
)
```

In this case, let's go with the decorator

In [1]:
from content_composer.registry import register_function
from typing import Any, Dict
import os
from loguru import logger
import asyncio
import requests

@register_function(
    identifier="perplexity_search",
    description="Perplexity search task",
    tags=["research", "search", "perplexity"],
    version="1.0.0",
    author="Your Name"
)
async def perplexity_search_task(inputs: Dict[str, Any]) -> Dict[str, Any]:
    """
    Performs a web search using Perplexity's API based on inputs.
    Expected inputs: 'search_query' (str), 'recency' (str, optional), 'mode' (str, optional).
    """
    search_query = inputs.get("search_query")
    recency = inputs.get("recency")  # Optional
    mode = inputs.get("mode")  # Optional, defaults to 'fast' in underlying logic

    if not search_query:
        return {"error": "'search_query' not provided for perplexity_search_task"}

    api_key = os.environ.get("PERPLEXITY_API_KEY")
    if not api_key:
        logger.error("PERPLEXITY_API_KEY not found in environment variables.")
        return {"error": "PERPLEXITY_API_KEY not set."}

    url = "https://api.perplexity.ai/chat/completions"
    pplx_model = "sonar-reasoning-pro" if mode == "expert" else "sonar-pro"

    messages = [
        {
            "role": "system",
            "content": "You are a helpful AI assistant.",  # Simplified system prompt for now, can be expanded
        },
        {
            "role": "user",
            "content": search_query,
        },
    ]
    payload = {
        "model": pplx_model,
        "messages": messages,
        "max_tokens": 3500,  # Made integer
        "temperature": 0.7,
        "top_p": 0.9,
        "return_citations": True,
        "return_images": False,
        "return_related_questions": False,
    }
    if recency:
        payload["search_recency_filter"] = recency

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    response = await asyncio.to_thread(
        requests.post, url, json=payload, headers=headers
    )
    response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)

    response_json = response.json()
    content = (
        response_json.get("choices", [{}])[0].get("message", {}).get("content")
    )

    if content is None:
        return {
            "error": "Perplexity API query failed to return content.",
            "details": response_json,
        }
    return {"content": content, "citations": response_json.get("citations", [])}


2025-06-20 17:28:58.481 | INFO     | content_composer.registry.registry:initialize:26 - Function registry initialized with 0 functions


In [2]:
from content_composer import parse_recipe, execute_workflow

# workflows can be defined in multiple formats, this example uses a json file
recipe = parse_recipe("./news_summary.json")
recipe

Recipe(name='AI News Summary via Perplexity', user_inputs=[UserInput(id='main_topic', label='Main Topic for Context (e.g., Generative AI)', type='string', description=None, default=None, required=True, literal_values=None), UserInput(id='research_instructions', label='Instructions for the research', type='text', description=None, default=None, required=True, literal_values=None), UserInput(id='summary_instructions', label='Instructions for the summary', type='text', description=None, default=None, required=True, literal_values=None), UserInput(id='recency', label='Recency', type='literal', description=None, default='week', required=True, literal_values=['day', 'week', 'month', 'year', 'any']), UserInput(id='mode', label='Mode of the news', type='literal', description=None, default='fast', required=True, literal_values=['fast', 'deep'])], nodes=[Node(id='fetch_ai_news', type=<NodeType.FUNCTION_TASK: 'function_task'>, description=None, prompt=None, model=None, function_identifier='perple

In [3]:
inputs = {"main_topic": "The future of AI and its impact on society", 
    "research_instructions": "Focus on the impact this will have on the job market", 
    "summary_instructions": "Summarize as if you are talking to a journlist", 
    "recency": "month", 
    "mode": "deep"}
outputs = await execute_workflow(
    recipe,
    inputs
)

In [4]:
outputs

{'summarize_news': '**News Summary: The Future of AI and Its Impact on Society**\n\nRecent developments in artificial intelligence (AI) are reshaping the job market, presenting both challenges and opportunities across various sectors. As AI technologies like ChatGPT become more prevalent, roles susceptible to automation—particularly in IT and knowledge work—are facing significant disruption. Job postings for high-exposure roles have seen a decline of up to 31%, with forecasts suggesting that AI could eliminate as much as half of all entry-level white-collar positions within the next five years, potentially pushing unemployment rates to between 10% and 20%.\n\nConversely, the demand for AI-related skills is rising, leading to higher wages for workers adept in these technologies. A recent report from PwC indicates that wages in AI-exposed industries are increasing at double the rate of those less affected, highlighting a wage premium for skilled professionals. This "AI-driven skills eart